In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
import sys, os, time
import pickle
import h5py
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tensorflow_addons.optimizers import CyclicalLearningRate

from utils import get_tf_datasets, fetch_callbacks, get_tf_dataset_sampled, quantile_loss_wrapper
from model import get_model

# from competition baseline
from base_code.helper import to_observed_matrix

In [ ]:
np.random.seed(42)
tf.keras.utils.set_random_seed(42)

# Load data and declare variables
- e.g. file paths, targets, etc.

In [ ]:
# adjust as needed
training_path = 'data/FullDataset/TrainingData'

In [ ]:
training_GT_path = os.path.join(training_path, 'Ground Truth Package')
GT_trace_path = os.path.join(training_GT_path, 'Tracedata.hdf5')

spectral_training_data = h5py.File(os.path.join(training_path,'SpectralData.hdf5'),"r")
aux_data = pd.read_csv(os.path.join(training_path,'AuxillaryTable.csv'))
soft_label_data = pd.read_csv(os.path.join(training_GT_path, 'FM_Parameter_Table.csv'))

trace_file = h5py.File(GT_trace_path)

if 'Unnamed: 0' in soft_label_data.columns:
    soft_label_data = soft_label_data.drop(['Unnamed: 0'], axis=1)

# assume this order?
target_labels = ['planet_radius','planet_temp','log_H2O','log_CO2','log_CO','log_CH4','log_NH3']
num_targets = len(target_labels)

target_scales_dict = {
    'planet_radius': [0.1, 3],
    'planet_temp': [0, 7000],
    'log_H2O': [-12, -1],
    'log_CO2': [-12, -1],
    'log_CO': [-12, -1],
    'log_CH4': [-12, -1],
    'log_NH3': [-12, -1]
}

target_scales_arr = np.array([target_scales_dict[l] for l in target_labels]) # safer, in case dict unordered
min_vals = target_scales_arr[:, 0]
max_vals = target_scales_arr[:, 1]

In [ ]:
spec_matrix_file = 'spec_matrix.npy'
if spec_matrix_file in os.listdir('data/'):
    print('opening existing spec matrix file')
    with open('data/spec_matrix.npy', 'rb') as f:
        spec_matrix = np.load(f)
    print('spectra shape:', spec_matrix.shape)
else:
    print('constructing spec matrix and saving to file')
    start_time = time.time()
    spec_matrix = to_observed_matrix(spectral_training_data, aux_data)
    with open('data/spec_matrix.npy', 'wb') as f:
        np.save(f, spec_matrix)
    print("finished after: {}, spectral matrix shape: {}".format(time.time() - start_time, spec_matrix.shape))

noise = spec_matrix[:, :, 2]
spectra = spec_matrix[:, :, 1]
wl_grid = spec_matrix[:, :, 0]
bin_width = spec_matrix[:, :, 3]

global_spectra_mean = np.mean(spectra)
global_spectra_std = np.std(spectra)
print(f'spectra mean: {global_spectra_mean}, std: {global_spectra_std}')

wl_channels = spectra.shape[1] # wavelength_channels

# Feature Engineering
- spectra stats
- ratio of planet mass to star mass
- star density
- planet_semimajor_axis: 
    - https://en.wikipedia.org/wiki/Orbital_period#Two_bodies_orbiting_each_other
- planet equilibrium temperature (planet_eqlbm_temp)
    - see equation (7): https://www.astro.princeton.edu/~strauss/FRS113/writeup3/

In [ ]:
spectra_rw_mean = spectra.mean(axis=1)
spectra_rw_std = spectra.std(axis=1)
spectra_rw_min = spectra.min(axis=1)
spectra_rw_max = spectra.max(axis=1)

aux_data['spectra_rw_mean'] = spectra_rw_mean
aux_data['spectra_rw_std'] = spectra_rw_std
aux_data['spectra_rw_min'] = spectra_rw_min
aux_data['spectra_rw_max'] = spectra_rw_max

In [ ]:
aux_data['planet_mass_kg/star_mass_kg'] = aux_data['planet_mass_kg']/aux_data['star_mass_kg']
aux_data['star_density'] = aux_data['star_mass_kg']*(4/3)*np.pi*(aux_data['star_radius_m']**3)

# technically this needs Albedo's constant. i.e. (1 - A)^(1/4)
aux_data['planet_eqlbm_temp'] = np.sqrt(aux_data['star_radius_m']/(2*aux_data['planet_distance']))*aux_data['star_temperature']

# Kepler's third law (technically this needs the graviational constant)
aux_data['planet_semimajor_axis'] = ((aux_data['star_mass_kg'] + aux_data['planet_mass_kg'])*((aux_data['planet_orbital_period']/(2*np.pi))**2))**(1/3)

### Plot of dstributions shows skew (optional)

In [ ]:
cols = [c for c in aux_data.columns if c != 'planet_ID']

num_cols = 2
num_rows = int(np.ceil(len(cols)/num_cols))

fig, axs = plt.subplots(num_rows, num_cols, figsize=(11, 4*num_rows))

for n in range(num_rows):
    for m in range(num_cols):
        col_idx = n*num_cols + m
        sns.histplot(aux_data[cols[col_idx]], kde=False, ax=axs[n, m], label=cols[col_idx])

### Apply log transfrom
- natural log, from <a href="https://numpy.org/doc/stable/reference/generated/numpy.log.html">numpy.log</a>

In [ ]:
cols = [c for c in aux_data.columns if c != 'planet_ID']
for c in cols:
    aux_data[c] = np.log(aux_data[c])

In [ ]:
cols = [c for c in aux_data.columns if c != 'planet_ID']

num_cols = 2
num_rows = int(np.ceil(len(cols)/num_cols))

fig, axs = plt.subplots(num_rows, num_cols, figsize=(11, 4*num_rows))

for n in range(num_rows):
    for m in range(num_cols):
        col_idx = n*num_cols + m
        sns.histplot(aux_data[cols[col_idx]], kde=False, ax=axs[n, m], label=cols[col_idx])

# Get which planets have trace data

In [ ]:
planet_desc_filename = 'planets_data_desc.csv'

if planet_desc_filename in os.listdir():
    print('reading from file')
    df_planet_has_data = pd.read_csv('planets_data_desc.csv')

else:
    print('building planet_data_desc')
    planet_list = [p for p in trace_file.keys()] 

    planet_data_existence = []
    for idx, pl in enumerate(planet_list):
        # print(trace_file[pl]['weights'].shape != ())
        has_data = trace_file[pl]['weights'].shape != ()
        planet_data_existence.append((pl, has_data))
        
    print(f'finished, total planets: {len(planet_data_existence)}')
    
    # convenience stuff
    df_planet_has_data = pd.DataFrame(planet_data_existence, columns=['planet_id_str', 'has_trace_data'])
    df_planet_has_data['planet_id'] = df_planet_has_data['planet_id_str'].str \
                                                                         .replace('Planet_train', '') \
                                                                         .astype(int)
    df_planet_has_data['planet_ID'] = df_planet_has_data['planet_id_str'].str.replace('Planet_', '')
    df_planet_has_data = df_planet_has_data.sort_values(by=['planet_id']).reset_index(drop=True)
    
    df_planet_has_data.to_csv(planet_desc_filename, index=False)
    
total_with_data = df_planet_has_data['has_trace_data'].sum()
print(f'has_data: {total_with_data}, no_data: {df_planet_has_data.shape[0] - total_with_data}')
df_planet_has_data.head()

In [ ]:
soft_d_has_trace = soft_label_data.merge(df_planet_has_data, on='planet_ID')
soft_d_has_trace.head()

# Split out test set
- 10% of planets with trace data?

In [ ]:
cols_for_split = ['planet_ID', 'planet_temp', 'planet_radius']

In [ ]:
train, test = train_test_split(soft_d_has_trace, 
                train_size=0.9,
                random_state=42)
print(f'train_shape: {train.shape}, val_shape: {test.shape}')
train.head()

In [ ]:
percent_test_with_trace = test['has_trace_data'].sum()/soft_d_has_trace['has_trace_data'].sum()
print(f'percent of test set that has trace data: {percent_test_with_trace:.2%}')

# Kfold training - Phase 1
- train on soft targets
- loss function of mse

In [ ]:
aux_columns = aux_data.columns
num_splits = 5
batch_size = 1024
train_repeat_count = 10
val_repeat_count = 5

# model parameters
n_feature_cols = None
num_added_features = len(aux_columns) - 1
num_spectra_features = wl_channels
loss = 'mse'
lr = 1e-3
epochs = 50

In [ ]:
kf = KFold(n_splits=num_splits, shuffle=True, random_state=42)
fold = 0
scalers = []

df_full_history = pd.DataFrame()
start_time = time.time()
for train_idx, val_idx in kf.split(soft_d_has_trace[cols_for_split], y=None, groups=None):
    fold_start_time = time.time()
    print('starting fold: {}'.format(fold))
    
    # get data
    train_aux_data = aux_data[aux_columns].iloc[train_idx]
    train_targets = soft_label_data.iloc[train_idx]
    train_spectra = spectra[train_idx]
    train_noise = noise[train_idx]
    
    val_aux_data = aux_data[aux_columns].iloc[val_idx]
    val_targets = soft_label_data.iloc[val_idx]
    val_spectra = spectra[val_idx]
    val_noise = noise[val_idx]
    
    # standardize aux_data and targets
    aux_data_ss = StandardScaler()
    std_train_aux_data = aux_data_ss.fit_transform(train_aux_data.drop(['planet_ID'], axis=1).values)
    std_train_targets = (train_targets.drop(['planet_ID'], axis=1)[target_labels].values - min_vals)/(max_vals - min_vals)

    std_val_aux_data = aux_data_ss.transform(val_aux_data.drop(['planet_ID'], axis=1).values)
    std_val_targets = (val_targets.drop(['planet_ID'], axis=1)[target_labels].values - min_vals)/(max_vals - min_vals)
    scalers.append(aux_data_ss)
    
    # get datasets
    train_dataset = get_tf_datasets(train_spectra, train_noise, std_train_aux_data, std_train_targets, 
                                    train_repeat_count, batch_size)
    val_dataset = get_tf_datasets(val_spectra, val_noise, std_val_aux_data, std_val_targets, 
                                    val_repeat_count, batch_size)
    
    # get and compile model
    model = get_model(loss, num_spectra_features, n_feature_cols, num_added_features, num_targets)
    model.compile(optimizer=tf.keras.optimizers.Adam(lr), loss='mse')
    
    # fit model
    weights_file = f'./weights/stage_1_mse_fold_{fold}_of_{num_splits}'
    print(f'...saving checkpoint to {weights_file}')
    callbacks = fetch_callbacks(epochs, batch_size, weights_file, patience=3)
    history = model.fit(train_dataset, 
                          validation_data=val_dataset,
                          epochs=epochs, 
                          shuffle=True,
                          callbacks=callbacks)
    
    # append history
    df_history = pd.DataFrame(history.history)
    df_history['fold'] = fold
    
    print('...finished training model with min val_loss: {}'.format(df_history['val_loss'].min()))
    df_full_history = pd.concat([df_full_history, df_history]).reset_index(drop=True)
    print('...finished fold: {}, after {} seconds'.format(fold, time.time() - fold_start_time))
    
    fold += 1

df_full_history.to_csv('histories/stage_1_mse_training_history.csv', index=False)
print(f'finished after {time.time() - start_time} seconds')

# Kfold training - phase 2

In [ ]:
loss = 'uniform_quantiles'
batch_size = 1024
train_repeat_count = 30
val_repeat_count = 5
epochs = 100 # too long?
num_splits = 5
quantile_percents = [0.05, 0.5, 0.95]

In [ ]:
kf = KFold(n_splits=num_splits, shuffle=True, random_state=42)
fold = 0
second_scalers = []

df_full_history = pd.DataFrame()
start_time = time.time()
for _train_idx, _val_idx in kf.split(train.loc[train['has_trace_data']]):
    train_idx = train.loc[train['has_trace_data']].iloc[_train_idx].index
    val_idx = train.loc[train['has_trace_data']].iloc[_val_idx].index

    fold_start_time = time.time()
    print('starting fold: {}'.format(fold))
    
    # get data
    train_aux_data = aux_data[aux_columns].iloc[train_idx]
    train_targets = soft_d_has_trace.iloc[train_idx]
    train_spectra = spectra[train_idx]
    train_noise = noise[train_idx]
    
    val_aux_data = aux_data[aux_columns].iloc[val_idx]
    val_targets = soft_label_data.iloc[val_idx]
    val_spectra = spectra[val_idx]
    val_noise = noise[val_idx]
    
    # standardize aux_data and targets
    aux_data_ss = StandardScaler()
    std_train_aux_data = aux_data_ss.fit_transform(train_aux_data.drop(['planet_ID'], axis=1).values)
    std_val_aux_data = aux_data_ss.transform(val_aux_data.drop(['planet_ID'], axis=1).values)
    second_scalers.append(aux_data_ss)
    with open(f'saved_objects/uniform_quantile_fold_{fold}_of_{num_splits}.pickle', 'wb') as f:
        pickle.dump(aux_data_ss, f)
    
    # get datasets
    train_dataset = get_tf_dataset_sampled(train_spectra, train_noise, std_train_aux_data, 
                                           train_targets['planet_ID'].values, train_repeat_count, 
                                           batch_size, trace_file, target_scales_arr, 
                                           target_labels=target_labels, split_outputs=True)
    val_dataset = get_tf_dataset_sampled(val_spectra, val_noise, std_val_aux_data, 
                                         val_targets['planet_ID'].values, val_repeat_count, 
                                         batch_size, trace_file, target_scales_arr, 
                                         target_labels=target_labels, split_outputs=True)
    
    # get model with prior stage weights
    weights_file = f'./weights/stage_1_mse_fold_{fold}_of_{num_splits}'
    print(f'...loading model from prior stage checkpoint: {weights_file}')
    prior_model = get_model('mse', num_spectra_features, n_feature_cols, num_added_features, num_targets, 
                            weights_file=weights_file)
    last_layer = prior_model.get_layer('final_dropout').output
    outputs = []
    for i in range(len(target_labels)):
        outputs.append(Dense(len(quantile_percents), activation=None, 
                             name='output_{}'.format(target_labels[i]))(last_layer))
    model = Model(inputs=prior_model.inputs, outputs=outputs)

    N = train_repeat_count*len(train_idx)
    iterations = N/batch_size
    step_size= 2 * iterations
    lr_schedule = CyclicalLearningRate(5e-5, 1e-3, step_size=step_size, scale_fn=lambda x: tf.pow(0.95, x))
    opt = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
    
    ckpt_weights_file = f'./weights/stage_2_uniform_quantiles_fold_{fold}_of_{num_splits}'
    print(f'...saving model checkpoints at {ckpt_weights_file}')
    callbacks = fetch_callbacks(epochs, batch_size, ckpt_weights_file, patience=4, ignore_lr_sched=True)
    
    # compile and fit
    loss_dict = {'output_{}'.format(t): quantile_loss_wrapper(quantile_percents) for t in target_labels}
    model.compile(optimizer=opt, loss=loss_dict)
    history = model.fit(train_dataset, 
                      validation_data=val_dataset,
                      epochs=epochs, 
                      shuffle=True,
                      callbacks=callbacks)
    
    # append history
    df_history = pd.DataFrame(history.history)
    df_history['fold'] = fold
    
    df_full_history = pd.concat([df_full_history, df_history]).reset_index(drop=True)
    print('...finished fold: {}, after {} seconds'.format(fold, time.time() - fold_start_time))
    
    fold += 1
    
    del(model)
    tf.keras.backend.clear_session()

df_full_history.to_csv('histories/stage_2_uniform_quantile_history.csv', index=False)
print(f'finished after {time.time() - start_time} seconds')